In [2]:
!pip install selenium
!pip install tqdm
!pip install requests
!pip install pandas
!pip install openpyxl
!pip install beautifulsoup4
!pip install lxml

  Using cached trio-0.31.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached websocket_client-1.9.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.7 MB 5.6 MB/s eta 0:00:02
   --------- ------------------------------ 2.4/9.7 MB 7.9 MB/s eta 0:00:01
   ----------------- ---------------------- 4.2/9.7 MB 8.4 MB/s eta 0:00:01
   --------------------------- ------------ 6.6/9.7 MB 8.8 MB/s eta 0:00:01
   ---------------

In [3]:
import os
import csv
import requests
import urllib
import zipfile
from tqdm import tqdm
from urllib.parse import urlparse
from concurrent.futures import ThreadPoolExecutor
import threading
import urllib3

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options

# Disable SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [18]:
BASE_URL = "https://qipedc.moet.gov.vn"

# # Sử dụng đường dẫn tuyệt đối để tránh vấn đề với Jupyter notebook
# project_root = r"C:\Users\PC\Documents\GitHub\sudo-visign"

# # Hoặc sử dụng đường dẫn tương đối từ thư mục notebook
project_root = os.path.join(os.path.dirname(os.path.abspath('')))

videos_dir = os.path.join(project_root, "dataset", "videos")
text_dir = os.path.join(project_root, "dataset", "text")
filtered_labels_path = os.path.join(project_root, "data", "cleaned_data.csv")
os.makedirs(videos_dir, exist_ok=True)  
os.makedirs(text_dir, exist_ok=True)

csv_path = os.path.join(text_dir, "label.csv")

csv_lock = threading.Lock()

In [21]:
def get_filtered_labels():
    labels = set()
    with open(filtered_labels_path, encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            labels.add(row["LABEL"].strip())
    return labels

def handle_recursive_scrapping(dict: list, driver):
    vids = WebDriverWait(driver = driver, timeout=3).until(
        expected_conditions.presence_of_all_elements_located((By.CSS_SELECTOR, "section:nth-of-type(2) > div:nth-of-type(2) > div:nth-of-type(1)"))
    )
    vids = driver.find_elements(By.CSS_SELECTOR, "section:nth-of-type(2) > div:nth-of-type(2) > div:nth-of-type(1) > a")
    for vid in vids:
        label = vid.find_element(By.CSS_SELECTOR, "p").text
        thumbs_url = vid.find_element(By.CSS_SELECTOR, "img").get_attribute("src")
        video_id = thumbs_url.replace("https://qipedc.moet.gov.vn/thumbs/", "").replace(".png", "")
        video_url = f"{BASE_URL}/videos/{video_id}.mp4"
        item = {'label': label, 'video_url': video_url}
        dict.append(item)

def csv_init():
    if not os.path.exists(csv_path):
        with open(csv_path, 'w', newline='', encoding= 'utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(["ID", "VIDEO", "LABEL"])

def add_to_csv(id, video, label):
    with csv_lock:
        with open(csv_path, 'a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow([id, video, label])

def download_video(video_data):
    video_url = video_data.get('video_url')
    label = video_data.get('label')
    filename = os.path.basename(urlparse(video_url).path)
    output_path = os.path.join(videos_dir, filename)
    if os.path.exists(output_path):
        print(f"Skip: {filename}")
        return
    try:
        print(f"Downloading: {filename}")
        # Disable SSL verification due to expired certificate
        response = requests.get(video_url, stream=True, verify=False)
        response.raise_for_status()
        total_size = int(response.headers.get('content-length', 0))
        with open(output_path, 'wb') as file, tqdm(
            desc=f"Progess {filename}",
            total=total_size,
            unit='B',
            unit_scale=True,
            unit_divisor=1024,
            ncols=100
        ) as bar:
            for data in response.iter_content(chunk_size=8192):
                size = file.write(data)
                bar.update(size)

        id = sum(1 for _ in open(csv_path, encoding='utf-8'))
        add_to_csv(id, filename, label)                 
        print(f"Completed: {filename}")
        print(f"Updated label.csv: {label}")

    except Exception as e:
        print(f"Error{filename}: {str(e)}")
        if os.path.exists(output_path):
            os.remove(output_path)  

def crawl_videos(filtered_labels):
    print("CRAWLING VIDEOS")
    options = Options()
    options.add_argument('--disable-dev-shm-usage')
    options.headless = False  # Change to True if you want headless mode
    service = Service()  # Uses system geckodriver (update path as needed)
    driver = webdriver.Firefox(service=service, options=options)
    videos = []
    try:
        driver.get("https://qipedc.moet.gov.vn/dictionary")
        print("Connected to dictionary website")
        
        handle_recursive_scrapping(videos, driver)

        for i in range(2, 5):
            id = i
            if i != 2: id = i + 1
            button = driver.find_element(By.CSS_SELECTOR, f"button:nth-of-type({id})")
            button.click()
            handle_recursive_scrapping(videos, driver)
            
        for i in range(5, 218):
            id = 6
            button = driver.find_element(By.CSS_SELECTOR, f"button:nth-of-type({id})")
            button.click()
            handle_recursive_scrapping(videos, driver)

        for i in range(218, 220):
            id = 6
            if i != 218: id = 7
            button = driver.find_element(By.CSS_SELECTOR, f"button:nth-of-type({id})")
            button.click()
            handle_recursive_scrapping(videos, driver)
    except Exception as e:
        print(f"Error: {e}")
    finally:
        driver.close()
    
    # Filter videos with labels in filtered_labels
    filtered_videos = [video for video in videos if video.get('label') in filtered_labels]
    return filtered_videos


In [22]:
filtered_labels = get_filtered_labels()
videos = crawl_videos(filtered_labels)
if videos:
    print(f"Found {len(videos)} videos in filtered topic list\n")

    print("STARTING DOWNLOAD VIDEOS")
    csv_init()
        
    with ThreadPoolExecutor(max_workers=3) as executor:
        executor.map(download_video, videos)
            
    print(f"DOWNLOAD COMPLETED {videos_dir}")
else:
    print("No videos found for given topic labels.")

CRAWLING VIDEOS
Connected to dictionary website
Found 309 videos in filtered topic list

STARTING DOWNLOAD VIDEOS
Downloading: D0032.mp4
Downloading: D0041.mp4
Downloading: D0093.mp4



Progess D0032.mp4:  25%|██████████▎                              | 152k/603k [00:00<00:00, 1.22MB/s]
Progess D0041.mp4: 100%|█████████████████████████████████████████| 575k/575k [00:00<00:00, 3.21MB/s]
Progess D0032.mp4:  65%|██████████████████████████▋              | 392k/603k [00:00<00:00, 1.70MB/s]
Progess D0032.mp4: 100%|█████████████████████████████████████████| 603k/603k [00:00<00:00, 2.40MB/s]




Completed: D0041.mp4
Updated label.csv: giỗ
Downloading: D0095.mp4
Completed: D0032.mp4
Updated label.csv: ai cho
Downloading: D0098.mp4


Progess D0093.mp4: 100%|███████████████████████████████████████| 1.17M/1.17M [00:00<00:00, 3.16MB/s]


Completed: D0093.mp4
Updated label.csv: đạo diễn
Downloading: D0119B.mp4


...

Completed: W02271.mp4
Updated label.csv: mùa thu
Downloading: W03272.mp4


Progess W02282.mp4:  45%|██████████████████▏                     | 344k/759k [00:00<00:00, 1.68MB/s]
Progess W02282.mp4: 100%|████████████████████████████████████████| 759k/759k [00:00<00:00, 2.46MB/s]


Progess W03165.mp4: 100%|████████████████████████████████████████| 590k/590k [00:00<00:00, 2.14MB/s]


Completed: W02282.mp4
Updated label.csv: mưa phùn
Downloading: W04068.mp4


Progess W03272.mp4: 100%|████████████████████████████████████████| 609k/609k [00:00<00:00, 1.93MB/s]
Progess W04068.mp4: 100%|████████████████████████████████████████| 327k/327k [00:00<00:00, 1.75MB/s]

Completed: W03165.mp4
Updated label.csv: cầu thang
Completed: W03272.mp4
Updated label.csv: thím
Completed: W04068.mp4
Updated label.csv: con trai
DOWNLOAD COMPLETED c:\Users\PC\Documents\GitHub\sudo-visign\dataset\videos
